In [2]:
#Black scholes hedge w simulated price paths 
import numpy as np
from scipy.stats import norm
import pandas as pd

def BlackScholesHedgeMC(S, K, sgm, r, T, callput, mu, n, m):
    dt = T/m
    sim = np.exp((mu - (0.5 * (sgm**2))) * (dt) 
                     + sgm * (np.random.standard_normal([m,n])) * np.sqrt(dt))
    sim = pd.DataFrame(sim)
    df = pd.DataFrame(np.array([S]*n)).T
    sim_2 = pd.concat([df, sim], ignore_index = True)
    price_path = sim_2.cumprod()
    
    Hedge_pnl_list = []
    
    for pp in range(0, n):
        t_list = []
        S = price_path.loc[:, pp]
        running_pnl = 0
        for i in range(1, len(S)):
            a = (T/(len(S)-1))*i
            d_1 = (np.log(S[i] / K) + (r + ((sgm**2)/2))*((T-a))) / (sgm * (np.sqrt((T-a))))
            d_2 = d_1 - (sgm * (np.sqrt((T-a))))

            b = (T/(len(S)-1)) * (i-1)
            d_1_1 = (np.log(S[i-1] / K) + (r + ((sgm**2)/2))*((T-b))) / (sgm * (np.sqrt((T-b))))
            d_2_2 = d_1_1 - (sgm * (np.sqrt((T-b))))

            if i < (len(S)):

                S_i = S[i] * (norm.cdf(d_1_1) - norm.cdf(d_1))
                K_i = K * np.exp(-r*(T-a)) * (norm.cdf(d_2_2) - norm.cdf(d_2))

                if callput == 1:
                    pnl = (S_i - K_i)
                elif callput == -1:
                    pnl = (K_i - S_i)

                running_pnl += (pnl * np.exp(r*(T-a)))

            elif i == (len(S)):
                S_i = S.iloc[i,1] * norm.cdf(d_1_1)
                K_i = K * norm.cdf(d_2_2)

                if callput == 1:
                    pnl = (S_i - K_i - max(0, S[i] - K))
                elif callput == -1:
                    pnl = (K_i - S_i - max(0,   K- S[i]))

                running_pnl += pnl
            
            
        Hedge_pnl_list.append(running_pnl)       
    #print (Hedge_pnl_list)
    PnL = np.array(Hedge_pnl_list)
    MC = PnL.mean()
    MCstd = np.sqrt(PnL.var()/n)
    
    return MC, MCstd 
            


if __name__ == '__main__':
    stock = float(input('Enter the underlying stock price: '))
    strike = float(input('Enter the strike price: '))
    sigma = float(input('Enter the volatility: '))
    interest = float(input('Enter continuously compounded interest rate: '))
    maturity = float(input('Enter the time to maturity: '))
    callput = int(input('Enter 1 for call or -1 for put option: '))
    mu = float(input('Enter the drift mu: '))
    n = int(input('Enter the number of simulations: '))
    m = int(input('Enter the number of time steps: '))
    MC, MCstd = BlackScholesHedgeMC(stock,strike,sigma,interest,maturity,callput,mu,n,m)
    print('The average simulated profit/loss is: ')
    print(MC)
    print('The 2 standard deviation confidence interval for the mean profit/loss is: ')
    print(MC-2*MCstd,MC+2*MCstd)

Enter the underlying stock price: 100
Enter the strike price: 100
Enter the volatility: 0.2
Enter continuously compounded interest rate: 0.05
Enter the time to maturity: 2
Enter 1 for call or -1 for put option: 1
Enter the drift mu: 0.01
Enter the number of simulations: 1000
Enter the number of time steps: 30
The average simulated profit/loss is: 
0.041462253024531645
The 2 standard deviation confidence interval for the mean profit/loss is: 
-0.06594637464779145 0.14887088069685472
